In [6]:
# import subprocess
# subprocess.run("ollama serve", shell=True)

In [7]:
import ollama


['qwen2.5:latest',
 'nemotron-mini:latest',
 'llama3.2:latest',
 'nomic-embed-text:latest']

In [7]:
# import ollama
# #Then everytime you want to chat
# response = ollama.chat(model='llama3.2', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#   },
# ])
# print(response['message']['content'])

In [2]:
import pandas as pd
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

In [3]:
from dotenv import load_dotenv
load_dotenv() # read local .env file
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [4]:
import streamlit as st
# from streamlit_extras.add_vertical_space import add_vertical_space
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os

In [5]:
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)


In [10]:
# embeddings = AzureOpenAIEmbeddings(
#     model="text-embedding-3-large",
#     deployment= 'genai-gpt-35-turbo',
#     # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
#     azure_endpoint=OPENAI_API_BASE, #"https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
#     api_key=OPENAI_API_KEY, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
#     openai_api_version=OPENAI_API_VERSION, # If not provided, will read env variable AZURE_OPENAI_API_VERSION
# )

In [6]:
import time

In [14]:
# # Function to process text data in chunks and add to the FAISS index
# def process_in_chunks(text_data, chunk_size, embeddings, index_name):
#     # Check if the index already exists
#     if os.path.exists(f"{index_name}.faiss") and os.path.exists(f"{index_name}.pkl"):
#         vectorstore = FAISS.load_local(index_name, embeddings)
#         print("Loaded existing FAISS index.")
#     else:
#         vectorstore = None
    
#     for i in range(0, len(text_data), chunk_size):
#         chunk = text_data[i:i + chunk_size]
#         try:
#             print(f"Processing chunk {i // chunk_size + 1}")
#             temp_vectorstore = FAISS.from_texts(chunk, embeddings)
            
#             if vectorstore:
#                 vectorstore.merge_from(temp_vectorstore)
#             else:
#                 vectorstore = temp_vectorstore
            
#             # Save the updated FAISS index after processing each chunk
#             vectorstore.save_local(index_name)
#             print(f"Chunk {i // chunk_size + 1} processed and index saved.")
            
#         except Exception as e:
#             print(f"Error processing chunk {i}: {e}")
        
#         time.sleep(1)  # Sleep to respect the rate limit
    
#     return vectorstore



In [7]:
# Initialize embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

# Load documents
loader = DirectoryLoader("./input-data", glob="**/*.txt")
print("Directory loaded")
documents = loader.load()

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1500,
    chunk_overlap=300,
    add_start_index=True,
)

# Split documents into chunks
texts = text_splitter.split_documents(documents)

Directory loaded


In [6]:
# Create FAISS index
vectorstore = FAISS.from_documents(texts, embeddings,)

print("FAISS index created successfully")

# Save the index locally
vectorstore.save_local("my_faiss_index")

print("FAISS index saved locally")

NameError: name 'texts' is not defined

In [8]:
# Load the saved index
loaded_vectorstore = FAISS.load_local("my_faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)



In [9]:
load_dotenv() # read local .env file
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [113]:
llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY,
                        openai_api_type=OPENAI_API_TYPE,
                        #deployment_name = 'genai-gpt-4-32k',
                        #model_name = 'gpt-4-32k')
                        deployment_name = 'genai-gpt-35-turbo',
                        model_name = 'gpt-4o',
                        temperature = 0)

In [11]:
# # Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based only on the following context and provide a meaningful response. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the answer, politely state that you don't have the specific information and suggest contacting the sales department for more details.

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template )

# # Modify the rag_chain to return the result instead of streaming
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
rag_chain.invoke("how to change my RFP affiliation")

'To change your RFP affiliation, you must submit an affiliation change request accompanied by documentation (such as an email) from the account requesting such. Please include account number, group to be affiliated with, and quarter to begin new affiliation. Alternatively, you can send the request to VISUS SalesInquir@its.jnj.com directly, ensuring that you include SAP, date of affiliation change, and group to be affiliated with. It is important to note that an account must join or have joined the requested RFP group on or before the last day of the first month of the quarter to qualify for a rebate. Affiliation change requests will be honored until the last day of the quarter.'

In [14]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import initialize_agent, AgentType, Tool
import pandas as pd

In [130]:
df = pd.read_csv(r'./dummy_CAR.csv').rename(columns=str.lower)
df_columns =df.columns

In [115]:
# df_agent = create_pandas_dataframe_agent(llm, df, verbose=True,allow_dangerous_code=True,#max_iterations = 1
                                        #  )

# Create embeddings for the vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

# Load the saved index
loaded_vectorstore = FAISS.load_local("my_faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based on the following context, including retrieved documents and relevant employee data when applicable. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the specific information, politely state that you don't have the answer and suggest contacting the sales department for further details.

CONTEXT: {context}

DATAFRAME INFO: {dataframe_info} dataframe columns = {df_columns}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)

In [152]:
def check_sap_in_roster(df: pd.DataFrame, sap_value: int) -> pd.DataFrame:
    """Check if a particular SAP value exists in the roster. and say if it exist also provide the name and address along with rfp_name"""
    return df[df['sap'] == sap_value][['sap','rfp_name_random',
       'rfp_location', 'name', 'address',]]


In [153]:
tools = [
    Tool(
    name="check_sap_in_roster",
    func=lambda x: check_sap_in_roster(df, int(x)),
    description="""
    Useful for checking if a specific SAP value exists in the roster dataframe.
    Input: A numeric SAP value (e.g., 100241).
    Output: If the SAP exists, it returns relevant details such as rfp_name, name, and address.
    If the SAP doesn't exist, it returns an empty DataFrame.
    include and summarise all the information returned by the tool
    """
),
    Tool(
        name="FAQ for sales Inquiries",
        func=retriever.get_relevant_documents,
       description="Use this tool for general sales inquiries, questions about the tracker, or when you can't find specific information about an account. Input should be the full question or description of the issue."
    ),


    
]


In [154]:
# Create custom agent prompt with friendly HR assistant context
agent_kwargs = {
    'prefix': '''You are a sales inquiry assistant specializing in rebates,
      market share, promotions, and payouts. Your primary tasks are:
1. Checking if SAP numbers exist in the roster
2. Answering general sales inquiries
3. Helping with issues related to finding accounts in the tracker

Follow these steps for each query:
1. Determine which tool is most appropriate for the query.
2. Use the selected tool and analyze its output.
3. If the tool doesn't provide a satisfactory answer, consider using the other tool or stating that you don't have the information.
4. Provide a clear and concise answer based on the tool's output.

Remember:
- For SAP number checks, use the "check_sap_in_roster" tool.
- For all other inquiries, including tracker issues, use the "FAQ_for_sales_inquiries" tool.
- If you can't find an answer, politely state that and suggest contacting the sales department for further assistance.

You have access to the following tools:''',
    'format_instructions': '''Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question''',
    'suffix': '''Begin!

Question: {input}
Thought: Let's approach this step-by-step:'''
}

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
    early_stopping_method="generate",
    max_iterations=3
)

`agent_scratchpad` should be a variable in prompt.input_variables. Did not find it, so adding it at the end.


In [155]:
# Custom function to handle agent execution
def execute_agent(agent, query):
    try:
        result = agent.run(query)
        return result
    except Exception as e:
        if "Could not parse LLM output" in str(e):
            return str(e).split("Human:")[0]  # Return the part before "Human:" if parsing error occurs
        else:
            return f"An error occurred: {str(e)}"

In [158]:
query = "does this exist in the roster 30776?"
# query = "I cant find my account in the tracker"
execute_agent(agent, query)



> Entering new AgentExecutor chain...
Action: check_sap_in_roster
Action Input: 30776
Observation:       sap rfp_name_random  rfp_location                         name  \
29  30776             ABB         30776  Sanchez, Moore and Thompson   

                address  
29  9502 Matthew Forest  
Thought:The SAP number 30776 exists in the roster and belongs to Sanchez, Moore and Thompson, located at 9502 Matthew Forest.
Final Answer: Yes, SAP number 30776 exists in the roster and belongs to Sanchez, Moore and Thompson, located at 9502 Matthew Forest.

> Finished chain.


'Yes, SAP number 30776 exists in the roster and belongs to Sanchez, Moore and Thompson, located at 9502 Matthew Forest.'

In [ ]:
# # Function to check if a question is related to structured data in the DataFrame
# def is_dataframe_question(query):
#     related_keywords = ['rebate', 'market share', 'promotion', 'payout']
#     return any(keyword in query.lower() for keyword in related_keywords)

# # Main chatbot logic
# def chatbot(query):
#     # Retrieve relevant documents from the vector store (always needed)
#     context = retriever.get_relevant_documents(query)
#     context_str = "\n".join([doc.page_content for doc in context])

#     # Check if the query is related to the pandas DataFrame
#     if is_dataframe_question(query):
#         # Query the pandas DataFrame agent
#         dataframe_info = df_agent.run(query)
#     else:
#         # If it's not related to the DataFrame, leave dataframe_info empty
#         dataframe_info = "No relevant structured data found."

#     # Create input for the LLM with both context and DataFrame info
#     final_input = template.format(context=context_str, dataframe_info=dataframe_info, question=query)

#     # Get response from the LLM
#     response = llm_for_pandas(final_input)
#     return response

In [33]:
# Function to check if a question is related to structured data in the DataFrame
# def is_dataframe_question(query):
#     related_keywords = ['rebate', 'market share', 'promotion', 'payout']
#     return any(keyword in query.lower() for keyword in related_keywords)

# Main chatbot logic
def chatbot(query):
    # Retrieve relevant documents from the vector store (always needed)
    context = retriever.get_relevant_documents(query)
    context_str = "\n".join([doc.page_content for doc in context])

    # # Check if the query is related to the pandas DataFrame
    # if is_dataframe_question(query):
        # Query the pandas DataFrame agent
    
    # Always run the pandas DataFrame agent query, with error handling
    try:
        dataframe_info = df_agent.run(query)
    except Exception as e:
        # Handle the error and provide a default response
        dataframe_info = f"No relevant structured data found in DataFrame"

    # Create input for the LLM with both context and DataFrame info
    final_input = template.format(context=context_str, dataframe_info=dataframe_info, df_columns= df_columns, question=query)

    # Get response from the LLM
    response = llm.invoke(final_input)
    return response

In [92]:
chatbot('Account not found on the RfP Tracker')



> Entering new AgentExecutor chain...
Thought: This is not a question, but a statement. I need more information on what to do with this statement.
Final Answer: N/A

> Finished chain.


AIMessage(content='An account will not appear on the RfP Tracker if it has $0 in sales for the current quarter or if it is not an active member of any RfP group. There are known issues when searching by account name in the RfP tracker, so please try searching by account number. If you still cannot find the account, please contact the sales department for further details.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 1741, 'total_tokens': 1819, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-efe83d48-fcee-4789-b72b-7631837e12cd-0')

##### Langgraph Implementation

In [85]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field

In [96]:
load_dotenv()  # Load environment variables
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [103]:
llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY,
                        openai_api_type=OPENAI_API_TYPE,
                        #deployment_name = 'genai-gpt-4-32k',
                        #model_name = 'gpt-4-32k')
                        deployment_name = 'genai-gpt-35-turbo',
                        model_name = 'gpt-4o')

In [99]:
from langchain_openai import ChatOpenAI

In [104]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "roster"] = Field(
        ...,
        description="Given a user question choose to route it to roster or a vectorstore.",
    )
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

NotImplementedError: 

In [102]:
# Prompt
system = """ou are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. 
Answer the question based on the following context,
 including retrieved documents and relevant employee data when applicable. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the specific information, 
politely state that you don't have the answer and suggest contacting the sales department for further details.
Use the vectorstore for questions on these topics. Otherwise, roster."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

print(question_router.invoke({"question": "Account not found on the RfP Tracker"}))

APIConnectionError: Connection error.